In [2]:
import os
from os import path
import xml.etree.ElementTree as ET
import cv2

import numpy as np
import shutil as sh

import matplotlib.pyplot as plt

from utils import pad_image_labels, display_image_and_box_list, display_image_and_box, xmlbox2str, listbox2str

import yaml

In [2]:
####### ANNOTATIONS in .TXT CREATION ############
#################################################

source = 'coffee_raw'

label_paths = path.join(source, "Etiquetas")
image_paths = path.join(source, "Imagenes")

count_not1024 = 0
total_images = 0

class_to_index = dict()

for label_path in os.listdir(label_paths):
    label_full_path = path.join(label_paths, label_path)
    img_name, _ = label_path.split('.')

    img_full_path = path.join(image_paths, img_name) + ".jpg"
    img = cv2.imread(img_full_path)

    #print(f"before {img.shape}")

    if img.shape[2] != 3:

        print(img_full_path)
        print(img.shape, img_full_path, img)
        #new_img = np.swapaxes(img.copy(), 0,2)#.swapaxes(1,2)
        #print(img.shape)
        assert cv2.imwrite(img_full_path, new_img) 
        img = new_img.copy()
        break

    img_true_height, img_true_width, depth = img.shape
    assert depth == 3, f"depth = {depth}"
    

    xml_tree = ET.parse(label_full_path)
    root = xml_tree.getroot()

    # Width and Height vary
    img_size = root.find("size")

    img_width = int(img_size.find("width").text)
    img_height = int(img_size.find("height").text)

    # Image size describe in the XML should be the same as true image size (not always the case)
    if img_true_width != img_width or img_true_height != img_true_height :
         print(f"""Width and height of the image and XML info differs :
               True image size: {img_true_width} {img_true_height}
               XML info : {img_width} {img_height}
               for img {label_full_path}""")
        
    img_width = img_true_width
    img_height = img_true_height

    # Counting images
    if img_width != 1024 or img_height != 1024:
        count_not1024 += 1
    total_images += 1

    bbox_str_path = img_name + ".txt"
    bbox_str_path = path.join(image_paths, bbox_str_path)

    annotations = root.findall("object")
    str_full = xmlbox2str(annotations=annotations,
                          img_width=img_width,
                          img_height=img_height)

    with open(bbox_str_path, "w") as f:
        f.write(str_full)


print(f"Number of images not 1024x1024 format :{count_not1024} / {total_images}")

Width and height of the image and XML info differs :
               True image size: 4128 2322
               XML info : 2322 4128
               for img coffee_raw/Etiquetas/1615379843027.xml
Width and height of the image and XML info differs :
               True image size: 4128 2322
               XML info : 2322 4128
               for img coffee_raw/Etiquetas/1615401570714.xml
Width and height of the image and XML info differs :
               True image size: 1024 1024
               XML info : 0 0
               for img coffee_raw/Etiquetas/1615328632428.xml
Number of images not 1024x1024 format :261 / 438


In [3]:
########### PAD IMAGES TO FIT ##################
################################################

# At the moment, images vary is size (sic.) so we pad them to have the same ratio (3/2)

new_size = (480,640)
count_image = 0

raw_images_dir = "coffee_raw/Imagenes/"
problematic_images = {"1615401570714","1615379843027"} # Labels are fucked up for those, ignore them

for i, image_path in enumerate(os.listdir(raw_images_dir)):

    if image_path[-3:] == "txt":
        continue

    if image_path[:-4] in problematic_images:
        continue
    
    image_full_path = path.join(raw_images_dir, image_path)
    
    current_img = cv2.imread(image_full_path)
    img_shape = current_img.shape

    #print(image_full_path, img_shape)
    
    label_path = image_full_path[:-3] + "txt"
    label_file = open(label_path)
    #display_image_and_box(current_img, label_file, "oldbbox")


    if image_path[:-4] == "1615385610373": # Don't know this one specifically had problem in labels, but whatever
        current_img = cv2.rotate(current_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

    #current_img = cv2.resize(current_img, dsize=new_size)
    padded_image, bbox_list = pad_image_labels(current_img, label_file)
    downsize_img = cv2.resize(padded_image, dsize=new_size)

    # Replace old image with padded and resized version
    cv2.imwrite(image_full_path, downsize_img)
    bbox_str = listbox2str(bbox_list=bbox_list)

    label_file.close()
    with open(label_path, "w") as f:
        f.write(bbox_str)

    #if img_shape[0] < img_shape[1] or image_path[:-4] in problematic_images:
    #    print(image_full_path, img_shape)
    #    display_image_and_box_list(downsize_img, bbox_list, f"do pad work{i}")
        #cv2.imwrite(f"test_downsize{i}.png",downsize_img)
        #plt.savefig(f"testout{i}.png")
        #break

    count_image += 1


print(f"Total images reformat is {count_image}")




Total images reformat is 436


In [4]:
#### SPLIT TRAIN VAL FOR ALL IMAGES ####
########################################

raw_images_dir = "coffee_raw/Imagenes/"

#count_image = 436

proportion_trainval = 0.80
n_train = int(count_image * proportion_trainval)

count_copied_image = 0

train_dir = "datasets/train"
val_dir = "datasets/val"

if not os.path.exists("datasets"):
    os.mkdir("datasets")
    os.mkdir(train_dir)
    os.mkdir(val_dir)

    os.mkdir(path.join(train_dir, "images"))
    os.mkdir(path.join(train_dir, "labels"))

    os.mkdir(path.join(val_dir, "images"))
    os.mkdir(path.join(val_dir, "labels"))



for image_path in os.listdir(raw_images_dir):
    
    # Ignore .txt
    if image_path[-3:] == "txt":
        continue

    image_full_path = path.join(raw_images_dir, image_path)

    current_img = cv2.imread(image_full_path)
    img_shape = current_img.shape

    #print(img_shape)
    width, height = img_shape[0], img_shape[1]

    # Ignore images greater than 1024
    if width != 640 or height != 480 :
        print(f"Ignoring {image_path}, because format is not 640x480, is {width} {height}")
        continue
        #raise NotImplementedError("Images should be 640x480")

    
    if count_copied_image < n_train:
        image_new_full_path = path.join(train_dir, "images", image_path)
        label_new_full_path = path.join(train_dir, "labels", image_path)

    else:
        image_new_full_path = path.join(val_dir, "images", image_path)
        label_new_full_path = path.join(val_dir, "labels", image_path)



    # Copy image AND label txt to path
    sh.copy(image_full_path, image_new_full_path)
    sh.copy(image_full_path[:-3]+"txt", label_new_full_path[:-3]+"txt")

    count_copied_image += 1

print("Finished Splitting")


Ignoring 1615401570714.jpg, because format is not 640x480, is 2322 4128
Ignoring 1615379843027.jpg, because format is not 640x480, is 2322 4128
Finished Splitting


In [4]:
wheet_yaml = dict(
    train ='train',
    val ='val',
    nc =1,
    names =["cafe verde"]
)

with open('coffee.yaml', 'w') as outfile:
    yaml.dump(wheet_yaml, outfile, default_flow_style=True)